In [1]:
import pandas as pd
train = pd.read_csv("/Users/chunyu/Downloads/kaggle/bag_words/all/labeledTrainData.tsv", \
                    header=0,delimiter="\t",quoting=3)
test = pd.read_csv( "/Users/chunyu/Downloads/kaggle/bag_words/all/testData.tsv", header=0, delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv( "/Users/chunyu/Downloads/kaggle/bag_words/all/unlabeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )
print "Read %d labeled train reviews, %d labeled test reviews, " \
 "and %d unlabeled reviews\n" % (train["review"].size,  
 test["review"].size, unlabeled_train["review"].size )

Read 25000 labeled train reviews, 25000 labeled test reviews, and 50000 unlabeled reviews



In [5]:
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
import nltk.data
import sys
reload(sys)
sys.setdefaultencoding("utf-8")
def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review,'lxml').get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip().decode("utf-8"))
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

sentences = []  # Initialize an empty list of sentences

print "Parsing sentences from training set"
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

print "Parsing sentences from unlabeled set"
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)
print sentences[0]

In [7]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 2       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print "Training model..."
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save("/Users/chunyu/Downloads/kaggle/bag_words/part2test/"+model_name)

In [10]:
model.doesnt_match("man woman child kitchen".split())
model.most_similar("queen")

[(u'princess', 0.6696798205375671),
 (u'bride', 0.6406296491622925),
 (u'eva', 0.6122051477432251),
 (u'showgirl', 0.5983843803405762),
 (u'belle', 0.5872655510902405),
 (u'maid', 0.5788412690162659),
 (u'mistress', 0.578643798828125),
 (u'duchess', 0.5775558948516846),
 (u'prince', 0.5771911144256592),
 (u'stepmother', 0.5764634609222412)]

In [12]:
#model = Word2Vec.load("/Users/chunyu/Downloads/kaggle/bag_words/part2test/300features_40minwords_10context")
model["flower"]

array([-8.17515608e-03, -7.50409951e-03, -9.94273797e-02, -1.95446312e-02,
        6.34358004e-02, -5.58519131e-03,  4.91627194e-02,  3.98150161e-02,
       -3.29741426e-02, -1.21019902e-02, -1.75933931e-02, -6.43056631e-02,
        3.22396718e-02,  7.76314884e-02, -4.23342511e-02, -1.86181988e-03,
       -2.73934640e-02, -4.47293185e-02, -1.09582804e-01, -8.42227265e-02,
       -1.38397187e-01, -1.83965191e-02,  8.49597752e-02, -3.46158370e-02,
       -2.13027000e-02,  1.73701178e-02, -2.24113781e-02, -9.20993984e-02,
       -4.54084482e-03, -2.88989600e-02, -9.37011931e-03,  9.65658575e-03,
       -1.66249536e-02,  2.09181793e-02, -5.22986613e-02,  5.40473275e-02,
        5.73482700e-02, -3.69713604e-02,  1.68560613e-02, -1.97697617e-02,
       -6.41237595e-04,  9.29972380e-02,  2.78033694e-04, -7.83479139e-02,
        5.83739467e-02,  1.70655269e-02, -1.68525837e-02, -7.83756468e-03,
        2.61202212e-02,  3.73180397e-02, -4.01766738e-03,  8.35683048e-02,
        2.04445105e-02, -

In [21]:
import numpy as np  # Make sure that numpy is imported

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    if nwords != 0.:
        featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 1000th review
       if counter%1000. == 0.:
           print "Review %d of %d" % (counter, len(reviews))
       # 
       # Call the function (defined above) that makes average feature vectors
       reviewFeatureVecs[counter] = makeFeatureVec(review, model, \
           num_features)
       #
       # Increment the counter
       counter = counter + 1
    
    return reviewFeatureVecs

# ****************************************************************
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word
# removal.
from gensim.models import Word2Vec
model = Word2Vec.load("/Users/chunyu/Downloads/kaggle/bag_words/part2test/300features_40minwords_10context")
clean_train_reviews = []
for review in train["review"]:
    clean_train_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))

trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )

print "Creating average feature vecs for test reviews"
clean_test_reviews = []
for review in test["review"]:
    clean_test_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))

testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features )

from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier( n_estimators = 100 )

print "Fitting a random forest to labeled training data..."
forest = forest.fit( trainDataVecs, train["sentiment"] )

# Test & extract results 
result = forest.predict( testDataVecs )

# Write the test results 
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv( "/Users/chunyu/Downloads/kaggle/bag_words/part2test/Word2Vec_AverageVectors.csv", index=False, quoting=3 )